In [1]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
from klepto.archives import dir_archive
import cv2
import time
import itertools
from scipy.io import savemat
import gc # to force remove objects' associated memory you deallocate using <<del var_name>>  or <<var_name = None>>

In [2]:
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
flow_speeds = ['fast', 'med', 'slow']
submergences = ['Deep', 'Intermed', 'Shallow']
colors = {'corals':'dodgerblue','canopy':'green', 'rocks':'orange', 'dunes':'brown'}
quantities = ['slope_Azi_angle', 'slope_x', 'slope_y'] # to save

In [3]:
# bed_idx = 3
# flow_idx = 2
# submergence_idx = 2
TEST = 2
BATCH_FRAMES =  75# for data acquired at 3 fp, total of 900, per subfolder 225 frames. 225/75 = 9

# # SET the data variables
# FLOW_SPEED = flow_speeds[flow_idx]
# SUBMERGENCE = submergences[submergence_idx]
# BEDFORM = BEDFORMS[bed_idx]
# print("{}_{}Flow_{}H_test{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST))

## PSD using 2D FFT

In [4]:
s=[np.arange(len(BEDFORMS)), np.arange(len(flow_speeds)), np.arange(len(submergences)) ]
for bed_idx,flow_idx,submergence_idx in list(itertools.product(*s)): 
    FLOW_SPEED = flow_speeds[flow_idx]
    SUBMERGENCE = submergences[submergence_idx]
    BEDFORM = BEDFORMS[bed_idx]
    print("---------------------*****{}_{}Flow_{}H****----------------------".format(BEDFORM, FLOW_SPEED, SUBMERGENCE))
    try:
        for which_quantity in [0, 1, 2]:
            QUANTITY = quantities[which_quantity]
            print("--------------QUANTITY{}: {}---------".format(which_quantity,QUANTITY))
            for PORTION in [1,2,3,4]: # 4 subfolders split into 4 portions for saving
                print("#####PORTION: {}#####".format(PORTION))
                accumulated_data = []
                for serial,i in enumerate([PORTION]):#range(1,5):
                    # choose the data
                    SUBFOLDER = i

                    # source data folders
                    data_foldername = "{}_{}Flow_{}H_test{}_subFolder{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST, SUBFOLDER)
                    print(data_foldername)
                    data_location =  '../TOSHIBA_Drive/saksham_polar_cam_FLIR'
                    save_location =  "../TOSHIBA_Drive/saksham_polar_cam_FLIR/klepto_bulk_data_saves/all_MATdata_{}_{}Flow_{}H_test{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST)
                    if not os.path.exists(save_location):
                        os.mkdir(save_location)
                        print("save folder location created.")
                    klepto_saved_data_origin = 'klepto_bulk_data_saves' 

                    # retrive the saved data
                    data_origin = os.path.join(data_location, klepto_saved_data_origin, data_foldername)
                    bed_data = {BEDFORM:{}}
                    tic = time.time()
                    assert os.path.exists(data_origin) == True
                    data = dir_archive(data_origin, {}, serialized=True, cached=True)
                    data.load("all_sx_maps", "all_sy_maps")

                    if which_quantity == 0:
                        # derive the quantity you want to save -- for preventing OOM error,
                        # do the cv2 processing in memory for half the frames, clear memory and repeat once
                        NUM_FRAMES = data['all_sy_maps'].shape[-1]

                        if which_quantity == 0:
                            _, bed_data[BEDFORM]['all_slopeAzi_maps']\
                                        = cv2.cartToPolar(data['all_sy_maps'][:,:,:NUM_FRAMES//2],\
                                                          data['all_sx_maps'][:,:,:NUM_FRAMES//2], \
                                                          angleInDegrees=True)    
                            # Accumulate the data-
                            accumulated_data.append(bed_data[BEDFORM]['all_slopeAzi_maps'])
                            # clear space
                            bed_data[BEDFORM]['all_slopeAzi_maps']  = None

                            # repeat but notice this time the indices have changed
                            _, bed_data[BEDFORM]['all_slopeAzi_maps']\
                                        = cv2.cartToPolar(data['all_sy_maps'][:,:,NUM_FRAMES//2:],\
                                                          data['all_sx_maps'][:,:,NUM_FRAMES//2:], \
                                                          angleInDegrees=True)  
                            # Accumulate the data-
                            accumulated_data.append(bed_data[BEDFORM]['all_slopeAzi_maps'])
                            # clear space
                            bed_data[BEDFORM]['all_slopeAzi_maps']  = None
                    elif which_quantity == 1:
                        accumulated_data.append(data['all_sx_maps'])
                    elif which_quantity == 2:
                        accumulated_data.append(data['all_sy_maps'])

                    toc = time.time()
                    print("time taken to process 1 subfolder of data  = ", toc-tic)

                    del bed_data, data
                    gc.collect()

                # concatenate all the data so far
                data = np.concatenate(accumulated_data, axis =2)
                accumulated_data = None

                # save the accumulated data
                dic = {QUANTITY:data,\
                       "PORTION":PORTION,\
                        'BEDFORM':BEDFORM, 'FLOW_SPEED':FLOW_SPEED,'SUBMERGENCE':SUBMERGENCE}

                filename = '{}_QUANT{}_PORTION{}_{}Flow_{}H_test{}.mat'.format(BEDFORM, QUANTITY, PORTION, FLOW_SPEED, SUBMERGENCE, TEST)
                save_dest = os.path.join(save_location, filename)
                savemat(save_dest,dic)
                del dic, data
                gc.collect()
    except:
        continue

---------------------*****corals_fastFlow_DeepH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
corals_fastFlow_DeepH_test2_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  43.43179726600647
#####PORTION: 2#####
corals_fastFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  43.291555881500244
#####PORTION: 3#####
corals_fastFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  43.7594850063324
#####PORTION: 4#####
corals_fastFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  44.27119708061218
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
corals_fastFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  45.30022192001343
#####PORTION: 2#####
corals_fastFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  55.30848693847656
#####PORTION: 3#####
corals_fastFlow_DeepH_test2_subFolder3


time taken to process 1 subfolder of data  =  37.53347945213318
--------------QUANTITY2: slope_y---------
#####PORTION: 1#####
corals_medFlow_IntermedH_test2_subFolder1
time taken to process 1 subfolder of data  =  37.60443711280823
#####PORTION: 2#####
corals_medFlow_IntermedH_test2_subFolder2
time taken to process 1 subfolder of data  =  37.56845760345459
#####PORTION: 3#####
corals_medFlow_IntermedH_test2_subFolder3
time taken to process 1 subfolder of data  =  38.14013147354126
#####PORTION: 4#####
corals_medFlow_IntermedH_test2_subFolder4
time taken to process 1 subfolder of data  =  37.53248047828674
---------------------*****corals_medFlow_ShallowH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
corals_medFlow_ShallowH_test2_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  45.101139545440674
#####PORTION: 2#####
corals_medFlow_ShallowH_test2_subFolder2
time taken to process 1 subfolder of data

#####PORTION: 4#####
canopy_fastFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  44.98941946029663
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
canopy_fastFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  38.9254105091095
#####PORTION: 2#####
canopy_fastFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  38.24348998069763
#####PORTION: 3#####
canopy_fastFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  38.90279579162598
#####PORTION: 4#####
canopy_fastFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  38.28799748420715
--------------QUANTITY2: slope_y---------
#####PORTION: 1#####
canopy_fastFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  38.96766972541809
#####PORTION: 2#####
canopy_fastFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  38.385071992874146
#####PORTION: 3#####
canopy_fastFlow_DeepH_test2

time taken to process 1 subfolder of data  =  37.69028949737549
---------------------*****canopy_medFlow_ShallowH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
canopy_medFlow_ShallowH_test2_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  41.570828676223755
#####PORTION: 2#####
canopy_medFlow_ShallowH_test2_subFolder2
time taken to process 1 subfolder of data  =  41.3856680393219
#####PORTION: 3#####
canopy_medFlow_ShallowH_test2_subFolder3
time taken to process 1 subfolder of data  =  43.31622672080994
#####PORTION: 4#####
canopy_medFlow_ShallowH_test2_subFolder4
time taken to process 1 subfolder of data  =  44.24760150909424
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
canopy_medFlow_ShallowH_test2_subFolder1
time taken to process 1 subfolder of data  =  34.99353647232056
#####PORTION: 2#####
canopy_medFlow_ShallowH_test2_subFolder2
time taken to process 1 subfolder of data  =  

rocks_fastFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  38.93393588066101
--------------QUANTITY2: slope_y---------
#####PORTION: 1#####
rocks_fastFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  41.071465253829956
#####PORTION: 2#####
rocks_fastFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  40.64829111099243
#####PORTION: 3#####
rocks_fastFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  38.908342599868774
#####PORTION: 4#####
rocks_fastFlow_DeepH_test2_subFolder4
---------------------*****rocks_fastFlow_IntermedH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
rocks_fastFlow_IntermedH_test2_subFolder1
save folder location created.
---------------------*****rocks_fastFlow_ShallowH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
rocks_fastFlow_ShallowH_test2_subFolder1
save folder locati

rocks_slowFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  45.37407183647156
#####PORTION: 4#####
rocks_slowFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  45.30207085609436
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
rocks_slowFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  39.968087673187256
#####PORTION: 2#####
rocks_slowFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  39.99673795700073
#####PORTION: 3#####
rocks_slowFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  38.882925510406494
#####PORTION: 4#####
rocks_slowFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  39.012187004089355
--------------QUANTITY2: slope_y---------
#####PORTION: 1#####
rocks_slowFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  40.106858015060425
#####PORTION: 2#####
rocks_slowFlow_DeepH_test2_subFolder2
time taken to

#####PORTION: 4#####
dunes_fastFlow_IntermedH_test2_subFolder4
time taken to process 1 subfolder of data  =  37.57184886932373
---------------------*****dunes_fastFlow_ShallowH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
dunes_fastFlow_ShallowH_test2_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  43.33034110069275
#####PORTION: 2#####
dunes_fastFlow_ShallowH_test2_subFolder2
time taken to process 1 subfolder of data  =  43.996233224868774
#####PORTION: 3#####
dunes_fastFlow_ShallowH_test2_subFolder3
time taken to process 1 subfolder of data  =  44.94701838493347
#####PORTION: 4#####
dunes_fastFlow_ShallowH_test2_subFolder4
time taken to process 1 subfolder of data  =  45.01743698120117
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
dunes_fastFlow_ShallowH_test2_subFolder1
time taken to process 1 subfolder of data  =  36.74836349487305
#####PORTION: 2#####
dunes_fastFlow_ShallowH

dunes_slowFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  37.400726556777954
--------------QUANTITY2: slope_y---------
#####PORTION: 1#####
dunes_slowFlow_DeepH_test2_subFolder1
time taken to process 1 subfolder of data  =  39.022536516189575
#####PORTION: 2#####
dunes_slowFlow_DeepH_test2_subFolder2
time taken to process 1 subfolder of data  =  39.57366895675659
#####PORTION: 3#####
dunes_slowFlow_DeepH_test2_subFolder3
time taken to process 1 subfolder of data  =  37.521026611328125
#####PORTION: 4#####
dunes_slowFlow_DeepH_test2_subFolder4
time taken to process 1 subfolder of data  =  38.3650016784668
---------------------*****dunes_slowFlow_IntermedH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
dunes_slowFlow_IntermedH_test2_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  44.589431285858154
#####PORTION: 2#####
dunes_slowFlow_IntermedH_test2_subFolder2
time taken to 